# Lab 2 Introduction

Execute the following cell to load slides for this part of the workshop. After loading, if you wish, you can click on the "Slide Show" icon at the bottom right of the slide to see the slides in fullscreen.

In [1]:
!pip install -q simpletransformers

You should consider upgrading via the '/usr/bin/python -m pip install --upgrade pip' command.


In [2]:
!pip install pandas

You should consider upgrading via the '/usr/bin/python -m pip install --upgrade pip' command.


In [3]:
!pip install -q datasets transformers[sentencepiece] simpletransformers

You should consider upgrading via the '/usr/bin/python -m pip install --upgrade pip' command.


In [4]:
!wget https://github.com/kobkrit/datasets/raw/main/AIFORTHAI-LST20Corpus.tar.gz


--2022-02-14 12:58:34--  https://github.com/kobkrit/datasets/raw/main/AIFORTHAI-LST20Corpus.tar.gz
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/kobkrit/datasets/main/AIFORTHAI-LST20Corpus.tar.gz [following]
--2022-02-14 12:58:34--  https://raw.githubusercontent.com/kobkrit/datasets/main/AIFORTHAI-LST20Corpus.tar.gz
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13551362 (13M) [application/octet-stream]
Saving to: ‘AIFORTHAI-LST20Corpus.tar.gz.1’

AIFORTHAI-LST20Corp 100%[===================>]  12.92M  --.-KB/s    in 0.06s   

2022-02-14 12:58:35 (226 MB/s) - ‘AIFORTHAI-LST20Corpus.tar.gz.1’ 

In [5]:
!tar -xvf AIFORTHAI-LST20Corpus.tar.gz

._LST20_Corpus
LST20_Corpus/
LST20_Corpus/._genres.txt
LST20_Corpus/genres.txt
LST20_Corpus/._.DS_Store
LST20_Corpus/.DS_Store
LST20_Corpus/._test
LST20_Corpus/test/
LST20_Corpus/._LST20 Annotation Guideline.pdf
LST20_Corpus/LST20 Annotation Guideline.pdf
LST20_Corpus/._AGREEMENT.txt
LST20_Corpus/AGREEMENT.txt
LST20_Corpus/._train
LST20_Corpus/train/
LST20_Corpus/._eval
LST20_Corpus/eval/
LST20_Corpus/._DESCRIPTION.txt
LST20_Corpus/DESCRIPTION.txt
LST20_Corpus/._LST20 Brief Specification.pdf
LST20_Corpus/LST20 Brief Specification.pdf
LST20_Corpus/eval/._T12289.txt
LST20_Corpus/eval/T12289.txt
LST20_Corpus/eval/._T13197.txt
LST20_Corpus/eval/T13197.txt
LST20_Corpus/eval/._T13183.txt
LST20_Corpus/eval/T13183.txt
LST20_Corpus/eval/._T12510.txt
LST20_Corpus/eval/T12510.txt
LST20_Corpus/eval/._T12276.txt
LST20_Corpus/eval/T12276.txt
LST20_Corpus/eval/._T12262.txt
LST20_Corpus/eval/T12262.txt
LST20_Corpus/eval/._T12504.txt
LST20_Corpus/eval/T12504.txt
LST20_Corpus/eval/._T13154.txt
LST20_Cor

In [6]:
from datasets import load_dataset
lst20 = load_dataset("lst20", data_dir="LST20_Corpus")

Using custom data configuration default-data_dir=LST20_Corpus
Reusing dataset lst20 (/root/.cache/huggingface/datasets/lst20/default-data_dir=LST20_Corpus/0.0.0/e1b2a921fb011578ab43ddbbf789f3c500d62cb2df8ae4ed4b60bae8e4c0d3ad)


  0%|          | 0/3 [00:00<?, ?it/s]

In [7]:
lst20

DatasetDict({
    train: Dataset({
        features: ['id', 'fname', 'tokens', 'pos_tags', 'ner_tags', 'clause_tags'],
        num_rows: 63310
    })
    validation: Dataset({
        features: ['id', 'fname', 'tokens', 'pos_tags', 'ner_tags', 'clause_tags'],
        num_rows: 5620
    })
    test: Dataset({
        features: ['id', 'fname', 'tokens', 'pos_tags', 'ner_tags', 'clause_tags'],
        num_rows: 5250
    })
})

In [8]:
!pip install torch

You should consider upgrading via the '/usr/bin/python -m pip install --upgrade pip' command.


In [9]:
import logging

import torch
import pandas as pd
from simpletransformers.ner import NERModel, NERArgs

logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

train_data = [
    [0, "Harry", "B-PER"],
    [0, "Potter", "I-PER"],
    [0, "was", "O"],
    [0, "a", "O"],
    [0, "student", "B-MISC"],
    [0, "at", "O"],
    [0, "Hogwarts", "B-LOC"],
    [1, "Albus", "B-PER"],
    [1, "Dumbledore", "I-PER"],
    [1, "founded", "O"],
    [1, "the", "O"],
    [1, "Order", "B-ORG"],
    [1, "of", "I-ORG"],
    [1, "the", "I-ORG"],
    [1, "Phoenix", "I-ORG"],
]
train_data = pd.DataFrame(
    train_data, columns=["sentence_id", "words", "labels"]
)

eval_data = [
    [0, "Sirius", "B-PER"],
    [0, "Black", "I-PER"],
    [0, "was", "O"],
    [0, "a", "O"],
    [0, "prisoner", "B-MISC"],
    [0, "at", "O"],
    [0, "Azkaban", "B-LOC"],
    [1, "Lord", "B-PER"],
    [1, "Voldemort", "I-PER"],
    [1, "founded", "O"],
    [1, "the", "O"],
    [1, "Death", "B-ORG"],
    [1, "Eaters", "I-ORG"],
]
eval_data = pd.DataFrame(
    eval_data, columns=["sentence_id", "words", "labels"]
)

In [10]:
# See tags from https://github.com/huggingface/datasets/blob/master/datasets/lst20/lst20.py
_POS_TAGS = ["NN", "VV", "PU", "CC", "PS", "AX", "AV", "FX", "NU", "AJ", "CL", "PR", "NG", "PA", "XX", "IJ"]
_NER_TAGS = [
        "O",
        "B_BRN",
        "B_DES",
        "B_DTM",
        "B_LOC",
        "B_MEA",
        "B_NUM",
        "B_ORG",
        "B_PER",
        "B_TRM",
        "B_TTL",
        "I_BRN",
        "I_DES",
        "I_DTM",
        "I_LOC",
        "I_MEA",
        "I_NUM",
        "I_ORG",
        "I_PER",
        "I_TRM",
        "I_TTL",
        "E_BRN",
        "E_DES",
        "E_DTM",
        "E_LOC",
        "E_MEA",
        "E_NUM",
        "E_ORG",
        "E_PER",
        "E_TRM",
        "E_TTL",
    ]
_CLAUSE_TAGS = ["O", "B_CLS", "I_CLS", "E_CLS"]

In [11]:
import pandas as pd
train_df = pd.DataFrame(lst20['train'])
validation_df = pd.DataFrame(lst20['validation'])
test_df = pd.DataFrame(lst20['test'])

In [12]:
def convert_to_simple_transformer_format(df, field_name, tags):
  sentence_id = []
  words = []
  labels = []

  #Limit at 1000 rows for speed.
  for (idx, r) in df[:1000].iterrows():
    # print(idx)
    for (i, t) in enumerate(r['tokens']):
      # print(i,t)
      sentence_id.append(idx)
      words.append(t)
      labels.append(tags[r[field_name][i]])

  return pd.DataFrame(
      {"sentence_id": sentence_id, "words": words, "labels": labels}
  )


In [13]:
train_ = convert_to_simple_transformer_format(train_df, "pos_tags", _POS_TAGS)

In [14]:
validation_ = convert_to_simple_transformer_format(validation_df, "pos_tags", _POS_TAGS)
test_ = convert_to_simple_transformer_format(test_df, "pos_tags", _POS_TAGS)

In [21]:
# Configure the model
model_args = NERArgs()
model_args.train_batch_size = 16
model_args.evaluate_during_training = True
model_args.overwrite_output_dir = False
model_args.num_train_epochs = 30

model = NERModel(
    "roberta", "xlm-roberta-large", args=model_args, use_cuda=torch.cuda.is_available(), labels=_POS_TAGS
)

# Train the model
model.train_model(train_, eval_data=validation_)

You are using a model of type xlm-roberta to instantiate a model of type roberta. This is not supported for all configurations of models and can yield errors.
Some weights of the model checkpoint at xlm-roberta-large were not used when initializing RobertaForTokenClassification: ['lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForTokenClassification were not initialized from the model 

  0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch:   0%|          | 0/30 [00:00<?, ?it/s]

Running Epoch 0 of 30:   0%|          | 0/63 [00:00<?, ?it/s]

...

INFO:simpletransformers.ner.ner_model: Converting to features started.


  0%|          | 0/2 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/125 [00:00<?, ?it/s]

/usr/local/lib/python3.8/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: VV seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.8/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: NN seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.8/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: AX seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.8/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: AV seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.8/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: PU seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.8/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: N

(1890,
 defaultdict(list,
             {'global_step': [63,
               126,
               189,
               252,
               315,
               378,
               441,
               504,
               567,
               630,
               693,
               756,
               819,
               882,
               945,
               1008,
               1071,
               1134,
               1197,
               1260,
               1323,
               1386,
               1449,
               1512,
               1575,
               1638,
               1701,
               1764,
               1827,
               1890],
              'train_loss': [0.5781707763671875,
               0.17562149465084076,
               0.12291000038385391,
               0.1145494133234024,
               0.08851264417171478,
               0.08278190344572067,
               0.0720001682639122,
               0.008722078055143356,
               0.07080715894699097,
        

In [22]:
# Make predictions with the model
predictions, raw_outputs = model.predict(["ประยุกธ์ เผย ลูกน้อง ไม่ เชื่อ พูด ตลก ได้ สามารถ แข่ง กับ โน๊ต อุดม เดี่ยว ไมโครโฟน เมื่อ วันที่ 26 กุมภาพันธ์ ที่ ตลาด หลักทรัพย์ แห่ง ประเทศไทย"])
predictions

INFO:simpletransformers.ner.ner_model: Converting to features started.


  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

[[{'ประยุกธ์': 'NN'},
  {'เผย': 'VV'},
  {'ลูกน้อง': 'NN'},
  {'ไม่': 'NG'},
  {'เชื่อ': 'VV'},
  {'พูด': 'VV'},
  {'ตลก': 'VV'},
  {'ได้': 'AX'},
  {'สามารถ': 'AX'},
  {'แข่ง': 'VV'},
  {'กับ': 'PS'},
  {'โน๊ต': 'NN'},
  {'อุดม': 'NN'},
  {'เดี่ยว': 'NN'},
  {'ไมโครโฟน': 'NN'},
  {'เมื่อ': 'PS'},
  {'วันที่': 'NN'},
  {'26': 'NU'},
  {'กุมภาพันธ์': 'NN'},
  {'ที่': 'PS'},
  {'ตลาด': 'NN'},
  {'หลักทรัพย์': 'NN'},
  {'แห่ง': 'PS'},
  {'ประเทศไทย': 'NN'}]]

In [23]:
result, model_outputs, preds_list = model.eval_model(test_)

INFO:simpletransformers.ner.ner_model: Converting to features started.


  0%|          | 0/2 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/125 [00:00<?, ?it/s]

/usr/local/lib/python3.8/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: NN seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.8/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: PU seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.8/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: VV seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.8/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: PS seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.8/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: AV seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.8/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: C

In [25]:
to_pred = ""
file1 = open('pos_test.txt', 'r')
Lines = file1.readlines()
 
count = 0
# Strips the newline character
for line in Lines:
    count += 1
    to_pred += line.strip() + ' '
    #print("Line{}: {}".format(count, line.strip()))

In [26]:
to_pred.replace('  ',' ')

'อย่างไรก็ตาม เครื่อง บิน แอร์บัส เอ 380 ได้ ถึง ที่ หมาย ที่ ท่าอากาศยาน เชียงใหม่ ใน เวลา 14.50 น. โดย ใช้ เวลา บิน ประมาณ 1 ชั่วโมง มูลนิธิสืบฯ พอใจ ฟื้น ป่าห้วยขาแข้ง มี เสือโคร่ง มาก อันดับ 2 โลก 1 กันยายน 2550 17.31 น. ประธาน มูลนิธิ สืบ นาคะเสถียร พอใจ ป่า ห้วยขาแข้ง ฟื้นตัว พบ เสือโคร่ง 2-3 ตัว ต่อ 100 ตาราง กิโลเมตร มาก อันดับ ที่ 2 ของ โลก ตั้ง เป้า 5 ปี ขึ้น อันดับ 1 แสดง ถึง ยัง มี สัตว์ และ พืช ชนิด อื่น ๆ อุดมสมบูรณ์ วอน รัฐ เข้าใจ ปรับ นโยบาย จับตา ค้าน กฎหมาย เปิด ป่า ( 1 ก.ย. ) นาง รตยา จันทรเทียร ประธาน มูลนิธิ สืบ นาคะเสถียร ให้ สัมภาษณ์ ถึง สถานการณ์ ป่า และ สัตว์ ป่า ใน ป่าห้วยขาแข้ง จ. อุทัยธานี ว่า จาก การ วิจัย ทาง วิชาการ ล่าสุด พบ ว่า ผืน ป่า ห้วยขาแข้ง ซึ่ง เป็น ผืน ป่า ขนาด ใหญ่ ด้าน ตะวันตก เป็น มรดก ทาง ธรรมชาติ ของ โลก พื้นที่ แรก ของ ไทย ตั้งแต่ ปี 2534 ขณะ นี้ มี เสือโคร่ง อาศัย อยู่ 2-3 ตัว ต่อ พื้นที่ 100 ตาราง กิโลเมตร หรือ รวม มี เสือโคร่ง ประมาณ 80 - 100 ตัว ถือ เป็น อันดับ 2 ของ โลก รอง จาก ผืน ป่า ไซบีเรีย แต่ ทาง เจ้าหน้าที่ ผู้ ดูแล ป่า ได้ ตั้

In [ ]:
predictions, raw_outputs = model.predict(to_pred)
predictions

INFO:simpletransformers.ner.ner_model: Converting to features started.


  0%|          | 0/61 [00:01<?, ?it/s]

Running Prediction:   0%|          | 0/57660 [00:00<?, ?it/s]